In [232]:
import pandas as pandas
import numpy as np
import sklearn as sk



In [233]:
df_mp = pandas.read_json(path_or_buf="file://localhost/Users/dtraverso/projects/nasa_nea/data/mpcorb_extended.json")
df_mp.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
0,2.977808,NaN,1801-2015,MPCLINUX,NaN,2457400.5,0.12,3.34,0000,2015-11-28,...,MPO370963,1.376113,1.277355,2.458235e+06,0,2.768112,0.075754,10.59166,0.214007,0.60
1,3.412843,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.11,4.13,0000,2015-10-16,...,MPO350795,1.312198,1.276541,2.456634e+06,0,2.772362,0.231024,34.84095,0.213515,0.58
2,3.353932,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,5.33,0000,2015-12-18,...,MPO370963,1.247217,1.297421,2.456854e+06,0,2.669763,0.256266,12.98987,0.225941,0.60
3,2.571675,NaN,1821-2015,MPCLINUX,NaN,2457400.5,0.32,3.20,0000,2015-12-15,...,MPO370963,1.171498,1.380322,2.456924e+06,0,2.361670,0.088922,7.14005,0.271566,0.60
4,3.066885,NaN,1845-2015,MPCLINUX,NaN,2457400.5,0.15,6.85,0000,2015-12-20,...,MPO370963,1.239938,1.319481,2.457419e+06,0,2.574181,0.191402,5.36846,0.238641,0.58


In [234]:
df_mp.count()

Aphelion_dist                         713581
Arc_length                            135951
Arc_years                             577630
Computer                              713581
Critical_list_numbered_object_flag       578
Epoch                                 713581
G                                     710026
H                                     710026
Hex_flags                             713581
Last_obs                              713581
M                                     713581
NEO_flag                               14160
Name                                   19857
Node                                  713581
Num_obs                               713564
Num_opps                              713581
Number                                464622
One_km_NEO_flag                         1345
One_opposition_object_flag            120707
Orbital_period                        713581
Other_desigs                           59989
PHA_flag                                1691
Peri      

In [235]:
df_nea = pandas.read_json(path_or_buf="file://localhost/Users/dtraverso/projects/nasa_nea/data/nea_extended.json")
df_nea.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
0,1.782516,NaN,1893-2014,MPCLINUX,NaN,2457400.5,0.46,11.16,1804,2014-12-22,...,MPO355095,0.692794,2.315305,2.457230e+06,0,1.457878,0.222679,10.82861,0.559915,0.66
1,4.083845,NaN,1911-2016,MPCLINUX,NaN,2457400.5,0.15,15.50,1804,2016-02-06,...,MPO365250,0.917080,1.305524,2.456825e+06,0,2.633253,0.550874,11.56459,0.230656,0.42
2,3.884757,NaN,1918-2015,MPCLINUX,NaN,2457400.5,-0.12,13.40,1804,2015-07-06,...,MPO365253,0.840627,1.344521,2.457835e+06,0,2.478760,0.567217,9.35748,0.252553,0.73
3,4.085337,NaN,1924-2015,MPCLINUX,NaN,2457400.5,0.30,9.45,1804,2015-11-27,...,MPO370975,0.951983,1.298833,2.457392e+06,0,2.663276,0.533951,26.69239,0.226767,0.46
4,2.755192,NaN,1932-2015,MPCW,NaN,2457400.5,0.15,17.70,1804,2015-09-18,...,MPO347525,0.777716,1.602716,2.456954e+06,0,1.919387,0.435455,11.87965,0.370647,1.25


In [236]:
df_nea.count()

Aphelion_dist                         14097
Arc_length                             9106
Arc_years                              4991
Computer                              14097
Critical_list_numbered_object_flag      488
Epoch                                 14097
G                                     14097
H                                     14097
Hex_flags                             14097
Last_obs                              14097
M                                     14097
NEO_flag                              13866
Name                                    153
Node                                  14097
Num_obs                               14097
Num_opps                              14097
Number                                 2148
One_km_NEO_flag                        1343
One_opposition_object_flag             7806
Orbital_period                        14097
Other_desigs                             44
PHA_flag                               1679
Peri                            

In [237]:
df_mp_dups = pd.concat([df_mp, df_nea])
df_mp_dups.count()

# Dropping duplicates based on columns that are highly probable to have unique values per-row
df_mp_pure = df_mp_dups.drop_duplicates(subset=['Peri', 'n'], keep=False)
df_mp_pure.count()

Aphelion_dist                         699486
Arc_length                            126847
Arc_years                             572639
Computer                              699486
Critical_list_numbered_object_flag       375
Epoch                                 699486
G                                     695931
H                                     695931
Hex_flags                             699486
Last_obs                              699486
M                                     699486
NEO_flag                                 121
Name                                   19704
Node                                  699486
Num_obs                               699469
Num_opps                              699486
Number                                462474
One_km_NEO_flag                            4
One_opposition_object_flag            112901
Orbital_period                        699486
Other_desigs                           59945
PHA_flag                                  12
Peri      

In [238]:
df_pha = df_nea[df_nea['PHA_flag'] == 1]
df_pha.head()

,Aphelion_dist,Arc_length,Arc_years,Computer,Critical_list_numbered_object_flag,Epoch,G,H,Hex_flags,Last_obs,...,Ref,Semilatus_rectum,Synodic_period,Tp,U,a,e,i,n,rms
5,1.969507,NaN,1949-2015,MPCW,NaN,2457400.5,0.15,16.90,9803,2015-07-31,...,MPO350811,0.170467,9.378662,2.457556e+06,0,1.078063,0.826894,22.82823,0.880517,1.08
7,1.663050,NaN,1951-2015,MPCW,NaN,2457400.5,0.15,15.60,9803,2015-12-20,...,MPO370982,0.552619,3.565810,2.457654e+06,0,1.245343,0.335415,13.33771,0.709203,1.81
10,2.293395,NaN,1957-2015,MPCLINUX,NaN,2457400.5,0.09,16.25,9803,2015-12-16,...,MPO370985,0.504657,2.277674,2.457538e+06,0,1.470215,0.559904,6.35478,0.552882,0.52
20,2.930639,NaN,1973-2015,MPCLINUX,NaN,2457400.5,0.15,15.20,9803,2015-02-24,...,MPO350815,0.512846,1.731485,2.457385e+06,1,1.776134,0.650011,39.83312,0.416381,0.45
26,3.307079,NaN,1936-2013,MPCLINUX,NaN,2457400.5,0.15,18.80,8803,2013-03-13,...,MPO256878,0.390520,1.638026,2.457188e+06,1,1.874944,0.763828,1.33041,0.383903,0.86


In [239]:
df_nea_pure = df_nea[df_nea['PHA_flag'].isnull()]

df_nea_pure.dtypes

Aphelion_dist                         float64
Arc_length                            float64
Arc_years                              object
Computer                               object
Critical_list_numbered_object_flag    float64
Epoch                                 float64
G                                     float64
H                                     float64
Hex_flags                              object
Last_obs                               object
M                                     float64
NEO_flag                              float64
Name                                   object
Node                                  float64
Num_obs                                 int64
Num_opps                                int64
Number                                 object
One_km_NEO_flag                       float64
One_opposition_object_flag            float64
Orbital_period                        float64
Other_desigs                           object
PHA_flag                          

In [240]:
print 'Master Set Count =', len(df_mp_pure.index)
print 'NEA Set Count =', len(df_nea_pure.index)
print 'PHA Set Count =', len(df_pha.index)

Master Set Count = 699486
NEA Set Count = 12418
PHA Set Count = 1679


In [241]:
df_mp_pure['Label'] = 2
df_nea_pure['Label'] = 1
df_pha['Label'] = 0


/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/dtraverso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [242]:
df_joined = pd.concat([df_mp_pure, df_nea_pure, df_pha])
df_joined.count()

Aphelion_dist                         713583
Arc_length                            135953
Arc_years                             577630
Computer                              713583
Critical_list_numbered_object_flag       863
Epoch                                 713583
G                                     710028
H                                     710028
Hex_flags                             713583
Last_obs                              713583
M                                     713583
NEO_flag                               13987
Name                                   19857
Node                                  713583
Num_obs                               713566
Num_opps                              713583
Number                                464622
One_km_NEO_flag                         1347
One_opposition_object_flag            120707
Orbital_period                        713583
Other_desigs                           59989
PHA_flag                                1691
Peri      

In [243]:
df_joined.dtypes

Aphelion_dist                         float64
Arc_length                            float64
Arc_years                              object
Computer                               object
Critical_list_numbered_object_flag    float64
Epoch                                 float64
G                                     float64
H                                     float64
Hex_flags                              object
Last_obs                               object
M                                     float64
NEO_flag                              float64
Name                                   object
Node                                  float64
Num_obs                               float64
Num_opps                                int64
Number                                 object
One_km_NEO_flag                       float64
One_opposition_object_flag            float64
Orbital_period                        float64
Other_desigs                           object
PHA_flag                          

In [244]:
df_joined = df_joined.copy().drop(['Hex_flags', 'Last_obs', 'Name', 'Number','Arc_years','Other_desigs',
                            'Perturbers', 'Computer','Perturbers_2', 'Principal_desig', 'Ref', 'U'],
                          axis=1)
#df_joined = df_joined.drop(['Hex_flags', 'Last_obs', 'Name', 'Number'])
#df_joined = df_joined.drop(['Arc_years','Other_desigs', 'Perturbers', 'Computer'])
#df_joined = df_joined.drop(['Perturbers_2', 'Principal_desig', 'Ref', 'U'])
#df_joined.drop('Hex_flags', inplace=True)
#df_joined.dtypes
df_joined.dtypes

Aphelion_dist                         float64
Arc_length                            float64
Critical_list_numbered_object_flag    float64
Epoch                                 float64
G                                     float64
H                                     float64
M                                     float64
NEO_flag                              float64
Node                                  float64
Num_obs                               float64
Num_opps                                int64
One_km_NEO_flag                       float64
One_opposition_object_flag            float64
Orbital_period                        float64
PHA_flag                              float64
Peri                                  float64
Perihelion_dist                       float64
Semilatus_rectum                      float64
Synodic_period                        float64
Tp                                    float64
a                                     float64
e                                 

In [245]:
# Extract the Label column as a Series from the master set
sr_label = df_joined['Label']

# Drop the Label column from the master data set
df_joined = df_joined.drop('Label', axis=1)

df_joined.columns

Index([u'Aphelion_dist', u'Arc_length', u'Critical_list_numbered_object_flag',
       u'Epoch', u'G', u'H', u'M', u'NEO_flag', u'Node', u'Num_obs',
       u'Num_opps', u'One_km_NEO_flag', u'One_opposition_object_flag',
       u'Orbital_period', u'PHA_flag', u'Peri', u'Perihelion_dist',
       u'Semilatus_rectum', u'Synodic_period', u'Tp', u'a', u'e', u'i', u'n',
       u'rms'],
      dtype='object')

In [246]:
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm

X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_joined, sr_label, test_size=.25, random_state=0)



In [247]:
print X_train.shape
print X_test.shape
print y_train.shape
print X_test.shape

(535187, 25)
(178396, 25)
(535187,)
(178396, 25)


In [248]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring="f1_weighted")
print scores

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').